In [1]:


import ray


ray.init(log_to_driver=False)

2023-06-16 08:51:33,762	INFO worker.py:1538 -- Started a local Ray instance.


Python version:,3.10.6
Ray version:,2.2.0


# Define Configs

In [2]:
from envs import register_env_dosage

total_workers = 10
num_envs_per_worker = 1

env_name = "dosing-v0"

register_env_dosage(env_name)

[0.29274884]
[0.1375 0.2    0.725  0.45  ]
1.5125000000000028
1
Current Heart Rate: 74.5
Current Sys Blood Pressure: 111.0
Current Dia Blood Pressure: 84.5
Current Temperature: 36.55
Current Dose Concentration: 0.0
[0.10978764]
[0.14875 0.23    0.7025  0.405  ]
1.4862500000000014
2
Current Heart Rate: 74.05
Current Sys Blood Pressure: 111.9
Current Dia Blood Pressure: 84.05
Current Temperature: 36.595
Current Dose Concentration: 0.0
[0.2437732]
[0.158875 0.257    0.68225  0.3645  ]
1.4626249999999998
3
Current Heart Rate: 73.645
Current Sys Blood Pressure: 112.71000000000001
Current Dia Blood Pressure: 83.645
Current Temperature: 36.6355
Current Dose Concentration: 0.0
[0.36154115]
[0.1679875 0.2813    0.664025  0.32805  ]
1.4413624999999974
4
Current Heart Rate: 73.28049999999999
Current Sys Blood Pressure: 113.43900000000001
Current Dia Blood Pressure: 83.28049999999999
Current Temperature: 36.67195
Current Dose Concentration: 0.0
[0.41999367]
[0.17618875 0.30317    0.6476225  0.2952

/home/hamza/PycharmProjects/example-medicine-delievery-system/venv/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [3]:
from ray.rllib.algorithms.ppo import PPOConfig

lstm_model = {"fcnet_hiddens": [32, 32, 32], "vf_share_layers": False, "use_lstm": True,
              "lstm_cell_size": 32, "max_seq_len": 100}
algo = "PPO"
config = (
    PPOConfig()
    .environment(env_name)
    .training(gamma=0.996, num_sgd_iter=3, sgd_minibatch_size=100, clip_param=0.1, lr=1e-3,
              train_batch_size=500,
              entropy_coeff=1e-3, vf_clip_param=10000)
    .resources(num_gpus=1, num_cpus_per_worker=1)
    .rollouts(num_rollout_workers=total_workers, num_envs_per_worker=num_envs_per_worker, enable_connectors=True, observation_filter='MeanStdFilter')
    .framework("torch")
    .training(
        model=lstm_model)
    .evaluation(evaluation_num_workers=1)
)


/home/hamza/PycharmProjects/example-medicine-delievery-system/venv/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [4]:
log_dir = "tmp/pipeline_logs"

# Train RL Agent

The model is trained here using the best config from the tune step. The best training checkpoint is then chosen for evaluation

In [ ]:
from components.train import train_env

train_results = train_env(
    algo=algo,
    config=config,
    log_dir=log_dir,
    iterations=20000,
    stop_reward_mean=1000,
    name="solver",
    checkpoint_frequency=5
)
best_checkpoint = train_results.get_best_result(metric="episode_reward_mean", mode="max").best_checkpoints[0]
best_checkpoint_path = best_checkpoint[0]._local_path

2023-06-16 08:51:38,743	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-16 08:51:38,744	WARNING algorithm_config.py:571 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


Trial name,agent_timesteps_total,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,experiment_id,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
PPO_dosing-v0_3e2ec_00000,500,"{'num_env_steps_sampled': 500, 'num_env_steps_trained': 500, 'num_agent_steps_sampled': 500, 'num_agent_steps_trained': 500}",{},2023-06-16_08-51-54,False,nan,{},nan,nan,nan,0,0,891a35c0e6f84709a293b4c19cfcc937,hamza-Legion-5-15ACH6H,"{'learner': {'default_policy': {'custom_metrics': {}, 'learner_stats': {'cur_kl_coeff': 0.20000000000000007, 'cur_lr': 0.0010000000000000005, 'total_loss': 1153.5116455078125, 'policy_loss': 0.00857753619008387, 'vf_loss': 1153.502840169271, 'vf_explained_var': -0.0075374007225036625, 'kl': 0.009642003098270171, 'entropy': 1.6753312667210898, 'entropy_coeff': 0.0010000000000000005}, 'model': {}, 'num_grad_updates_lifetime': 8.0, 'diff_num_grad_updates_vs_sampler_policy': 7.0}}, 'num_env_steps_sampled': 500, 'num_env_steps_trained': 500, 'num_agent_steps_sampled': 500, 'num_agent_steps_trained': 500}",1,192.168.0.97,500,500,500,500,500,500,0,10,0,0,500,"{'cpu_util_percent': 38.8, 'ram_util_percent': 89.0}",171667,{},{},{},{},"{'episode_reward_max': nan, 'episode_reward_min': nan, 'episode_reward_mean': nan, 'episode_len_mean': nan, 'episode_media': {}, 'episodes_this_iter': 0, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [], 'episode_lengths': []}, 'sampler_perf': {}, 'num_faulty_episodes': 0}",0.439922,0.439922,0.439922,"{'training_iteration_time_ms': 418.25, 'load_time_ms': 6.351, 'load_throughput': 78721.922, 'learn_time_ms': 220.772, 'learn_throughput': 2264.782, 'synch_weights_time_ms': 4.01}",1686898314,0,500,1,3e2ec_00000,12.0164
